# STAR PERFORMERS

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import xmltodict
from pprint import pprint
import os
import csv
from api_keys import z_keys


In [ ]:

# #config
# from api_keys import api_key
# from api_keys import g_Api

# # # Import API key
# # import api_keys

# # Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# # Output File (CSV)
# output_data_file = "output_data/cities.csv"

# # Range of latitudes and longitudes
# lat_range = (-90, 90)
# lng_range = (-180, 180)
# import openweathermapy.core as owm

In [ ]:
ny18_df = pd.read_excel("/Users/JBagga/workspace/project_test/2018NYSales.xls")

In [ ]:
len(ny18_df)

In [ ]:
ny18_df.describe()

In [ ]:
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].str.split(' ')
ny18_df


In [ ]:
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].apply(lambda x: '+'.join(x))
# # ny18_df['ADDRESS'].str.join('+')
# ny18_df['ADDRESS'].str=ny18_df['ADDRESS'].str.replace(', ','+')
ny18_df

In [ ]:
# &citystatezip={ny18_Update['ZIP CODE']}

ny18_df['combo1'] = ny18_df['ADDRESS'] + "&citystatezip="  
# ,ny18_df['ZIP CODE'])
ny18_df['combo1']

In [ ]:
ny18_df['ZIP CODE']

In [ ]:
ny18_df['combo2']=ny18_df['combo1']+ny18_df['ZIP CODE'].map(str)
ny18_df

# ny18_df['ADDRESS'] = ny18_df['ADDRESS'].apply(lambda x: '+'.join(x))

In [ ]:
ny18_df_mod = pd.DataFrame(ny18_df)

In [ ]:
len(ny18_df_mod)

In [ ]:
# Removing home prices less than 100K to avoid a bias after preliminary research on what these lower sale prices rep.
# like: Internal family sale transfer at $0, garage sale etc.

ny18_Update = ny18_df_mod[ny18_df_mod['SALE PRICE'] > 100000]

In [ ]:
len(ny18_Update)

In [ ]:
ny18_Update.head()

In [ ]:
# from urllib.parse import urlparse
# ny18_Update['address_combo'] = urlparse.quote_plus(ny18_Update['ADDRESS'])

In [ ]:
# urllib.parse.quote_plus

In [ ]:
response = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address=635+west+42+street&citystatezip=10036&rentzestimate=true")

In [ ]:
response.text

In [ ]:
print(response)

In [ ]:
#Using BeautifulSoup to convert XML text to pretty format
soup = BeautifulSoup(response.text)
print(soup.prettify())
# soup.find_all

In [ ]:
# response['results']['result']['zpid']
#.keys() GIVES you the Current Level Keys in a Dictionary

doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']
# doc.values()
len(doc)

In [ ]:
def need_rent_estimate(address):
    return (address.get('rentzestimate')) and not(address.get('lastSoldPrice'))

In [ ]:
def need_sale_estimate(address):
    return (not(address.get('rentzestimate')) and (address.get('lastSoldPrice')))

In [ ]:
def get_rest_estimate(address):
    
    rent = address['rentzestimate']['amount']['#text']
#     sqft11 = address['finishedSqFt']
    address11 = address['address']['street']
    zip11=address['address']['zipcode']
    var = 2
    return pd.Series({'rent':rent, 'address11':address11,'zip11':zip11,'var':var})

In [ ]:
def get_other_rent_estimate(address):
    sold = address['lastSoldPrice']['#text']
    sold_date=address['lastSoldDate']
#     zestimate=address['zestimate']['amount']['#text']
#     sqft11 = address['finishedSqFt']
    address11 = address['address']['street']
    zip11=address['address']['zipcode']
    var = 1
    return pd.Series({'sold':sold, 'var':var}) 

In [ ]:
ny18_Update.columns

In [ ]:
# # block =[]
# # lot=[]
# # ny_sale_amt =[]
# # ny_sale_date=[]
# series = []


# for row,zipcode in zip(ny18_Update['ADDRESS'],ny18_Update['ZIP CODE']):
#     print(row)
#     response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={row}&citystatezip={zipcode}&rentzestimate=true")
# #     print(response_main)
# #     soup = BeautifulSoup(response_main.text)
# #     print(soup.prettify())
#     block=ny18_Update['BLOCK']
#     lot=ny18_Update['LOT']
#     sale_amt=ny18_Update['SALE PRICE']
#     sale_date=ny18_Update['SALE DATE']
                                 
#     doc = xmltodict.parse(response_main.text)['SearchResults:searchresults']['response']['results']['result']                             
#     time.sleep(.5)
#     i=0
#     print(len(doc))                             

#     while i<len(doc):
#         curdoc = doc[i]
#         print(curdoc)

#         if need_rent_estimate(curdoc):
#             new_series = get_rest_estimate(curdoc)

#         elif need_sale_estimate(curdoc):
#             new_series = get_other_rent_estimate(curdoc)      
#         i+=1

#         series.append(new_series)

#     df = pd.concat(series, 1,sort=True).T

In [ ]:
block =[]
lot=[]
ny_sale_amt =[]
ny_sale_date=[]
series = []
curdoc={}

for row in ny18_Update['combo2']:
    try:
        print(row)
        response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={row}&rentzestimate=true")
    #     response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={row}&citystatezip={ny18_Update['ZIP CODE']}&rentzestimate=true")
    #     print(response_main)
    #     soup = BeautifulSoup(response_main.text)
    #     print(soup.prettify())
        block=ny18_Update['BLOCK']
        lot=ny18_Update['LOT']
        sale_amt=ny18_Update['SALE PRICE']
        sale_date=ny18_Update['SALE DATE']

        doc = xmltodict.parse(response_main.text)['SearchResults:searchresults']['response']['results']['result']   
        print(doc)
        time.sleep(.1)
        i=0


        while i < len(doc):
            curdoc = doc[i]
            print(curdoc)

            if need_rent_estimate(curdoc):
                new_series = get_rest_estimate(curdoc)

            elif need_sale_estimate(curdoc):
                new_series = get_other_rent_estimate(curdoc)      
            i+=1

            series.append(new_series)

        df = pd.concat(series, 1,sort=True).T
    except KeyError:
        print(f'{curdoc}')

In [ ]:
# block =[]
# lot=[]
# ny_sale_amt =[]
# ny_sale_date=[]
# series = []


# for row in ny18_Update['ADDRESS']:
#     print(row)
#     response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={row}&citystatezip={ny18_Update['ZIP CODE']}&rentzestimate=true")
#     print(response_main)
#     soup = BeautifulSoup(response_main.text)
#     print(soup.prettify())
#     block=ny18_Update['BLOCK']
#     lot=ny18_Update['LOT']
#     sale_amt=ny18_Update['SALE PRICE']
#     sale_date=ny18_Update['SALE DATE']
                                 
#     doc = xmltodict.parse(response_main.text)['SearchResults:searchresults']['response']['results']['result']                             
#     time.sleep(.1)
#     i=0
                                 

#     while i<len(doc):
#         curdoc = doc[i]
#         print(curdoc)

#         if need_rent_estimate(curdoc):
#             new_series = get_rest_estimate(curdoc)

#         elif need_sale_estimate(curdoc):
#             new_series = get_other_rent_estimate(curdoc)      
#         i+=1

#         series.append(new_series)

#     df = pd.concat(series, 1).T

In [ ]:
# xmltodict.parse(response_main.text)['SearchResults:searchresults'].keys()

In [ ]:
# response_main = requests.get("https://www.zillow.com/webservice/GetDeepSearchResults.htm?\
# zws-id={z_keys}&address=\
# west+42+street&citystatezip=10036&rentzestimate=true")